# Query & look up data

The ORMs in LaminDB are Django Models and you can use [Django's full query capability](https://docs.djangoproject.com/en/stable/topics/db/queries/).

LaminDB also has `select` statements that are based on SQLAlchemy/SQLModel's way of writing queries, and closer to SQL itself.

(LaminDB was based on SQLAlchemy/SQLModel until v0.41.2 and might re-introduce compatibilty in the future.)

In [ ]:
# initialize a test instance for this notebook
!lamin delete myobjects
!lamin init --storage ./myobjects

In [ ]:
import lamindb as ln

In [ ]:
ln.track()

In [ ]:
# save some dummy files
ln.save(ln.File("index.md"))
ln.save(ln.File(ln.dev.datasets.df_iris(), name="iris"))
ln.save(ln.File(ln.dev.datasets.file_fastq()));

## Basic select statements

In [ ]:
user_handles = ln.User.lookup(field="handle")

With auto-complete, we find a user:

In [ ]:
user_handles.testuser1

Use it on one of the fields:

In [ ]:
user = ln.select(ln.User, handle=user_handles.testuser1).one()

In [ ]:
user

Query all files created by that user:

In [ ]:
ln.select(ln.File, created_by=user).df()

To access the query results encoded in a select statement (a extended Django `QuerySet` object), execute it with one of

- `.df()`: A pandas `DataFrame` with each record stored as a row.
- `.all()`: A django `QuerySet`.
- `.one()`: Exactly one record. Will raise an error if there is none.
- `.one_or_none()`: Either one record or `None` if there is no query result.

For example:

In [ ]:
ln.select(ln.File, created_by=user).all()[:3]

## Relationships

Rather than writing explicit joins as in SQLAlchemy and SQL itself, Django has a double-under-score syntax to select based on related tables.

In [ ]:
ln.select(ln.File, run__created_by__handle__startswith="testuse").df()

The query above selects all files based on filtering on the users who ran the generating notebook.

(Under the hood, in the SQL database, it's joining the file table with the run and the user table.)



## Field comparators

Django supports about [two dozen field comparators](https://docs.djangoproject.com/en/stable/ref/models/querysets/#field-lookups) `field__comparator=value`, called "lookup type".

Here are some of them.

### and

In [ ]:
ln.select(ln.File, suffix=".md", created_by=user).df()

### less than/ greater than

Or subset to files greater than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
ln.select(ln.File, created_by=user, size__lt=1e4).df()

### or

In [ ]:
from django.db.models import Q

ln.select(ln.File).filter(Q(suffix=".md") | Q(suffix=".fastq.gz")).df()

### in

In [ ]:
ln.select(ln.File, suffix__in=[".md", ".fastq.gz"]).df()

### order by

In [ ]:
ln.select(ln.File).order_by("-created_at").df()

### contains

In [ ]:
ln.select(ln.Transform, name__contains="lookup").df()

### startswith

In [ ]:
ln.select(ln.Transform, name__startswith="Query").df()

In [ ]:
!lamin delete myobjects